# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [19]:
#Install the dependencies
!pip install dlt[duckdb]

Defaulting to user installation because normal site-packages is not writeable


### Q1

In [8]:

limit = 5
generator = square_root_generator(limit)

value=0

for sqrt_value in generator:
    value+=sqrt_value

print(value)

8.382332347441762


> Answer: 8.382

### Q2

In [9]:
limit = 13
generator = square_root_generator(limit)

value=[]

for sqrt_value in generator:
     value.append(sqrt_value)

value[12]

3.605551275463989

> Answer: 3.605

### Q3

#### Create pipeline

In [10]:
import dlt

gen_pipeline=dlt.pipeline(destination='duckdb',dataset_name='gen_one')

gen_pipeline

#### First generator load

In [11]:
info=gen_pipeline.run(
    people_1(),
    table_name='people',
    write_disposition='replace'
)

info

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x000001D61B834B90>, metrics={'1707460853.349079': [{'started_at': DateTime(2024, 2, 9, 6, 40, 53, 776597, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 9, 6, 40, 54, 209358, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:///e:\\Development\\Git_Repos\\DEZ-2024-Homeworks\\Workshop_1\\dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='gen_one', loads_ids=['1707460853.349079'], load_packages=[LoadPackageInfo(load_id='1707460853.349079', package_path='C:\\Users\\saifk\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1707460853.349079', state='loaded', schema=Schema dlt_ipykernel_launcher at 2019102905344, schema_update={'people': {'name': 'people', 'columns': {'id': {'name': 'id', 'data_type': 'bigint'

#### Checking data after first generator load

In [12]:
import duckdb

conn = duckdb.connect(f"{gen_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{gen_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print('\n\n\n All data:')
display(conn.sql("select * from people"))


print('\n\n\n Cumulative age of all people:')
display(conn.sql("select sum(age) cumulative_age from people"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 All data:


┌───────┬──────────┬───────┬─────────┬───────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │   _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar      │    varchar     │
├───────┼──────────┼───────┼─────────┼───────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707460853.349079 │ 4X3zkoE+mtPNxg │
│     2 │ Person_2 │    27 │ City_A  │ 1707460853.349079 │ acew6cUl+YESDg │
│     3 │ Person_3 │    28 │ City_A  │ 1707460853.349079 │ 0Sa386wztERX/w │
│     4 │ Person_4 │    29 │ City_A  │ 1707460853.349079 │ o0urT/SSjWzqiA │
│     5 │ Person_5 │    30 │ City_A  │ 1707460853.349079 │ yrFH/6/7mtgU/Q │
└───────┴──────────┴───────┴─────────┴───────────────────┴────────────────┘




 Cumulative age of all people:


┌────────────────┐
│ cumulative_age │
│     int128     │
├────────────────┤
│            140 │
└────────────────┘

### Second generator load and append

In [13]:
info2=gen_pipeline.run(
    people_2(),
    table_name='people',
    write_disposition='append'
)

info2

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x000001D61B834B90>, metrics={'1707460857.7760403': [{'started_at': DateTime(2024, 2, 9, 6, 40, 58, 193595, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 9, 6, 40, 58, 502407, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:///e:\\Development\\Git_Repos\\DEZ-2024-Homeworks\\Workshop_1\\dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='gen_one', loads_ids=['1707460857.7760403'], load_packages=[LoadPackageInfo(load_id='1707460857.7760403', package_path='C:\\Users\\saifk\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1707460857.7760403', state='loaded', schema=Schema dlt_ipykernel_launcher at 2019103323920, schema_update={'people': {'name': 'people', 'columns': {'occupation': {'name': 'occupation', '

#### Checking data after loading secodn generator

In [14]:
print('\n\n\n All data:')
display(conn.sql("select * from people"))


print('\n\n\n Cumulative age of all people:')
display(conn.sql("select sum(age) cumulative_age from people"))




 All data:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707460853.349079  │ 4X3zkoE+mtPNxg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707460853.349079  │ acew6cUl+YESDg │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707460853.349079  │ 0Sa386wztERX/w │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707460853.349079  │ o0urT/SSjWzqiA │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707460853.349079  │ yrFH/6/7mtgU/Q │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707460857.7760403 │ aUISf706t84VTw │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707460857.7760403 │ kT1I1+BFBHUMWw │ Job_4      │
│     5 │ 




 Cumulative age of all people:


┌────────────────┐
│ cumulative_age │
│     int128     │
├────────────────┤
│            353 │
└────────────────┘

> Answer: 353

### Q4

#### Load data from both generator and merge

In [15]:
info3=gen_pipeline.run(
    people_1(),
    table_name='people_merged',
    write_disposition='replace',
    primary_key='id'
)

print(info3)

info4=gen_pipeline.run(
    people_2(),
    table_name='people_merged',
    write_disposition='merge',
    primary_key='id'
)

print(info4)


Pipeline dlt_ipykernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset gen_one
The duckdb destination used duckdb:///e:\Development\Git_Repos\DEZ-2024-Homeworks\Workshop_1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707460862.8530672 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.52 seconds
1 load package(s) were loaded to destination duckdb and into dataset gen_one
The duckdb destination used duckdb:///e:\Development\Git_Repos\DEZ-2024-Homeworks\Workshop_1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707460863.6139798 is LOADED and contains no failed jobs


#### Check the tables

In [16]:
print('\n\n\n All data:')
display(conn.sql("select * from people_merged"))


print('\n\n\n Cumulative age of all people:')
display(conn.sql("select sum(age) cumulative_age from people_merged"))




 All data:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707460862.8530672 │ PtCzrGhnFOuVww │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707460862.8530672 │ vyreobiAyG6Y5w │ NULL       │
│     7 │ Person_7 │    37 │ City_B  │ 1707460863.6139798 │ DbVOpibnpv5sOA │ Job_7      │
│     4 │ Person_4 │    34 │ City_B  │ 1707460863.6139798 │ kbfKPTBK9YVIbA │ Job_4      │
│     8 │ Person_8 │    38 │ City_B  │ 1707460863.6139798 │ 7PL1kWSfF1vgUA │ Job_8      │
│     3 │ Person_3 │    33 │ City_B  │ 1707460863.6139798 │ PO5xPzupqULKiQ │ Job_3      │
│     6 │ Person_6 │    36 │ City_B  │ 1707460863.6139798 │ nP5ZVLstlXHGlw │ Job_6      │
│     5 │ 




 Cumulative age of all people:


┌────────────────┐
│ cumulative_age │
│     int128     │
├────────────────┤
│            266 │
└────────────────┘

> Answer: 266